In [ ]:
%config IPCompleter.use_jedi = False
%config InlineBackend.figure_format='svg'
%matplotlib inline

In [ ]:
import os
import scipy
import urllib
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from scipy.stats import norm, multivariate_normal

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

seed = 1234
np.random.seed(seed)
plt.style.use("seaborn")

Hello Kagglers! This is going to be a series on understanding **Denoising Diffuison Probabilistic Models** or **DDPMs** for short. In this notebook I will talk about the **Gaussian Distribution** and its importance in understanding the **Denoising Diffusion Models**. Not only in the context of DDPMs but every Data Scientist/ML engineer should also be aware of Gaussian distribution and its implications. We will cover the following topics in this notebook:

* **Univariate Normal Distribution**
* **Multivariate Normal**
* **Covariance and Covariance Matrix**
* **Isotropic Gaussian**
* **Conditional Gaussian**
* **Convolution of PDFs**
* **Forward process in DDPMs and how all of the above is related to it**


We will cover the maths behind these concepts and will code them along the way. IMO the maths looks much scarier when presented in equations, but it becomes simple once you look at the code implementation. So, don't be scared by the greek symbols used in the equations. Try to undersatnd the code. It will help you understand the concepts better. Also, most of the visualizations presented in this notebook are interactive. Without further ado, let's start!

# 1. Normal Distribution

Also known as **Gaussian** Distribution, the normal distribution is one of the most widely used continuous probability distributions. The notation, $\mathcal{N}(\mu, \sigma^{2})$ , is commonly used to refer to a normal distribution. 

The parameter $\mu$ represents the **mean** or **expected value** of the distribution, while the parameter $\sigma$ is its **standard deviation**. The **variance** of the distribution is $\sigma^{2}$

For a real-valued random variable `x`, the probability density function is defined as:
$$
p(x; \mu, \sigma^{2}) = \frac{1}{\sigma \sqrt{2 \pi}} exp({\frac{-1}{2}(\frac {x - \mu}{\sigma})^{2}}) \tag {1}
$$


# 1.1 Univariate Normal Distribution

When we deal with only one random variable, e.g. `x` in the above equation, we call the distribution a univariate normal distribution. Let's code up the above equation and take an example of a univariate normal distribution.

In [ ]:
def get_univariate_normal(x, mu, sigma):
    """Generates a PDF for a Univariate Normal Distribution.

    Args:
        x: Vector of values sorted in ascending order
        mu: Mean of the Gaussian
        sigma: Standard Deviation of the Gaussian
    Returns:
        A PDF
    """
    return 1 / (sigma * np.sqrt(2 * np.pi)) * (np.exp(-(x - mu)**2 / (2 * sigma**2)))


# Mean of the distribution
mu = 0.0

# Standard deviation(SD for short)
sigma = 1.0

# Generate a linspace for a random variable x
num_samples = 100
x = np.linspace(-3*sigma + mu, 3*sigma + mu, num=num_samples)

# Plot the value against the PDF
fig = go.Figure(
    data=go.Scatter(
        x=x,
        y=get_univariate_normal(x, mu=mu, sigma=sigma),
        line=dict(width=3,color="black"),
        fill="tonexty",
        fillcolor="skyblue",
    )
)

fig.add_annotation(x=mu, y=-0.001, text="Mean",showarrow=True, arrowhead=2)
fig.add_vline(x=mu, line_width=3, line_dash="dash", line_color="green")
fig.update_layout(
    {
        "title": {
            'text': f"Univariate Gaussian Distribution <br> μ: {mu}, σ\u00b2: {sigma**2}",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(size=14)
        },
        "xaxis": {"title": "X"},
        "yaxis": {"title": "Probability Density"},
        "margin": dict(l=0, r=0, b=0, t=50)
    }
)

fig.show()

Moving the mean value shifts the distribution from right to left while changing sigma affects the shape of the distribution. As the value of sigma increases, the curve gets more and more flat. Let's see that in action.

In [ ]:
# Moving the mean shifts the distribution to
# right/left while increasing/decresasing the
# variance makes the curve more flat/pointy respectively
# Let's take an example to see this in action

# Random variable x
x = np.linspace(-10.0, 10.0, 100)

# Combination of mu and sigma. The first value
# of any tuple represents mu while the second value
# represents sigma here.
mu_sigma_combos = [
    [(0, 1), (-3, 1), (3, 1)],
    [(0, 1), (0, 2), (0, 4)],
]

# Line colors and widths to be used
# for different combinations
colors = ["red", "black", "blue"]
widths = [1, 2, 3]
subtitles = ["Varying μ", "Varying σ"]

# Plot
_, ax = plt.subplots(1, 2, sharex=False, sharey=False, figsize=(12, 5), tight_layout=True)
for i, elem in enumerate(mu_sigma_combos):
    legend = []
    mus = set()
    for j, comb in enumerate(elem):
        mu, sigma = comb
        mus.add(mu)
        legend.append(f"μ: {mu}, σ\u00b2: {sigma**2}")
        ax[i].plot(x, get_univariate_normal(x, mu, sigma), linewidth=widths[j], c=colors[j])
        ax[i].tick_params(labelbottom=True)
    
    ax[i].set_title(subtitles[i])
    ax[i].legend(legend, loc="upper right")
    
    for mu in mus:
        ax[i].axvline(x=mu, color="green", linestyle="--")

plt.suptitle("Univariate Normal Distribution")
plt.show()

# 1.2 Multivariate Normal Distribution

Multivariate normal distribution is the extension of univariate normal distribution to the case where we deal with a real-valued **vector input** instead of a single real-valued random variable. Like in the univariate case, the multivariate normal distribution has associated parameters $\mu$ representing the **mean vector** and $\Sigma$ representing the **covariance matrix**.

This is the probability density function for multivariate case:
$$
p(x; \mu, \Sigma) = \frac{1}{(2\pi)^{n/2} \ \vert{\Sigma}\vert^{1/2}} \ exp \ (\frac{-1}{2}(x - \mu)^T \Sigma^{-1}(x - \mu))  \tag{2}
$$

* Random Variable $X = [x_{1}, \ x_{2}, \ x_{3}...]$ is a `D`-dimensional vector
* Mean $\mu = [\mu_{1}, \ \mu_{2}, \ \mu_{3}...]$ is a `D`-dimensional vector
* Covariance Matrix $\Sigma$ is a `D X D` dimensional matrix


If the vector `X` is 2-dimensional, then that distribution is also known as **Bivariate Normal Distribution**.

**Note:** We need two dimensions to visualize a univariate Gaussian, and three dimesnions to visualize a bivariate Gaussian. Hence for visualization purposes, we will show everything with a *bivariate* Gaussian, which then, can be extended to *multivariate* cases. Let's visualize a bivariate Gaussian. We will use `scipy.stats.multivariate_normal` to generate the probability density.

In [ ]:
def get_multivariate_normal(
    mu,
    cov,
    sample=True,
    sample_size=None,
    seed=None,
    gen_pdf=False,
    pos=None
):
    """Builds a multivariate Gaussian Distribution.
    
    Given the mean vector and the covariance matrix,
    this function builds a multivariate Gaussian
    distribution. You can sample from this distribution,
    and generate probability density for given positions.
    
    Args:
        mu: Mean vector representing the mean values for
            the random variables
        cov: Covariance Matrix
        sample (bool): If sampling is required
        sample_size: Only applicable if sampling is required.
            Number of samples to extract from the distribution
        seed: Random seed to be passed for distribution, and sampling
        gen_pdf(bool): Whether to generate probability density
        pos: Applicable only if density is generated. Values for which
            density is generated
    Returns:
        1. A Multivariate distribution
        2. Sampled data if `sample` is set to True else `None`
        3. Probability Density if `gen_pdf` is set to True else `None`
    """
    
    # 1. Multivariate distribution from given mu and cov
    dist = multivariate_normal(mean=mu, cov=cov, seed=seed)
    
    # 2. If sampling is required
    if sample:
        samples = dist.rvs(size=sample_size, random_state=seed)
    else:
        samples = None
    
    # 3. If density is required
    if gen_pdf:
        if pos is None:
            raise ValueError("`pos` is required for generating density!")
        else:
            pdf = dist.pdf(pos)
    else:
        pdf = None
        
    return dist, samples, pdf



# Mean of the random variables X1 and X2
mu_x1, mu_x2 = 0, 0

# Standard deviation for the random variables X1 and X2
sigma_x1, sigma_x2 = 1, 1

# Positions for which probability density is
# to be generated
x1, x2 = np.mgrid[
    (-3.0 * sigma_x1 + mu_x1):(3.0 * sigma_x1 + mu_x1): 0.1,
    (-3.0 * sigma_x2 + mu_x2):(3.0 * sigma_x2 + mu_x2): 0.1
]
pos = np.dstack((x1, x2))

# Mean vector
mu = [mu_x1, mu_x2]

# Covariance between the two random variables
cov_x1x2 = 0

# Covariance Matrix for our bivariate Gaussian distribution
cov = [[sigma_x1**2, cov_x1x2], [cov_x1x2, sigma_x2**2]]

# Build distribution generate density
sample = get_multivariate_normal(
        mu=mu,
        cov=cov,
        sample=False,
        seed=seed,
        gen_pdf=True,
        pos=pos
    )


# Plot the bivariate normal density
fig = go.Figure(
    go.Surface(
        z=sample[2],
        x=x1,
        y=x2,
        colorscale='Viridis',
        showscale=False
    )
)
fig.update_layout(
    {
        "title": dict(
            text="Bivariate Distribution",
            y=0.95,
            x=0.5,
            xanchor="center",
            yanchor="top",
            font=dict(size=12)
        ), 
        "scene":dict(
            xaxis=dict(title='x1'),
            yaxis=dict(title='x2'),
            zaxis=dict(title='Probability density')
        ),
        "xaxis": dict(title="Values"),
        "yaxis": dict(title="Probability Density"),
    }
)
fig.show()

That's an interactive plot. You can zoom in/out, and rotate the surface to see what the Gaussians look like. When you hover over the density, you will see black lines generating two univariate normals, one for `x1` and another for `x2`


# 2. Covariance

In the last section, we introduced the term `covariance matrix`. Before we move on to the next section, it is important to understand the concept of covariance. 

Let's say `X` and `Y` are two random variables. Covariance of `X` and `Y` is defined as:

$$
Cov(X, Y) = E[(X - E(X))(Y - E(Y))] = E[XY] - E[X]E[Y]  \tag{3}
$$

i.e. the covariance is the expected value of the product of their deviations from their individual expected values.


So, what does **covariance** tell us? A lot...

1. Covariance gives a sense of how much two random variables as well their scales are **linearly** related
2. Covariance captures only linear dependence and gives no information about other kind of relationships
3. If the sign of the covariance is positive, then both variables tend to take on relatively high values simultaneously. If the sign of the covariance is negative, then one variable tends to take on a relatively high value at the times that the other takes on a relatively low value and vice versa.
4. If two variables `X` and `Y` are **independent**, then $Cov(X, Y)= 0$ but the reverse isn't true. Why? Because covariance doesn't take into account non-linear relationships

# 2.1 Covariance Matrix

If `X` is a `N`-dimensional vector i.e. $X = [x_{1}, \ x_{2}, \ x_{3}...x_{n}]$, the covariance matrix is a `N`X`N` matrix defined as:

$$
\begin{bmatrix}
Cov(x_{1}x_{1}) & \cdots & Cov(x_{1}x_{n}) \\
\vdots & \ddots & \vdots \\
Cov(x_{n}x_{1}) & \cdots & Cov(x_{n}x_{n})
\end{bmatrix}
$$

Each entry $(i, j)$ in this matrix defines the covariance of two random variables of the vector. Also:

$$
Cov(x_{i}x_{j}) = Var(x_{i}) \ \ \ \{ i=j\}
$$

Let's take an example to showcase zero covariance, positive covariance, and negative covariance between two random variables `x1` and `x2`

In [ ]:
# Mean of the random variables X1 and X2
mu_x1, mu_x2 = 0, 0

# Standard deviation of the random variables X1 and X2
sigma_x1, sigma_x2 = 1, 1

# Number of samples to extract
sample_size = 2000

# Positions for which probability density is
# to be generated
x1, x2 = np.mgrid[
    (-3.0 * sigma_x1 + mu_x1):(3.0 * sigma_x1 + mu_x1): 0.1,
    (-3.0 * sigma_x2 + mu_x2):(3.0 * sigma_x2 + mu_x2): 0.1
]
pos = np.dstack((x1, x2))

# Mean vector
mu = [mu_x1, mu_x2]


# Case 1: Zero Covariance
cov_x1x2 = 0
zero_cov = [[sigma_x1**2, cov_x1x2], [cov_x1x2, sigma_x2**2]]
# Build distribution, sample and generate density
zero_cov_res = get_multivariate_normal(
        mu=mu,
        cov=zero_cov,
        sample=True,
        sample_size=sample_size,
        seed=seed,
        gen_pdf=True,
        pos=pos
    )


# Case 2: Positive Covarinace
cov_x1x2 = 0.9
pos_cov = [[sigma_x1**2, cov_x1x2], [cov_x1x2, sigma_x2**2]]
# Build distribution, sample and generate density
pos_cov_res = get_multivariate_normal(
        mu=mu,
        cov=pos_cov,
        sample=True,
        sample_size=sample_size,
        seed=seed,
        gen_pdf=True,
        pos=pos
    )


# Case 3: Negative Covarinace
cov_x1x2 = -0.9
neg_cov = [[sigma_x1**2, cov_x1x2], [cov_x1x2, sigma_x2**2]]
# Build distribution, sample and generate density
neg_cov_res = get_multivariate_normal(
        mu=mu,
        cov=neg_cov,
        sample=True,
        sample_size=sample_size,
        seed=seed,
        gen_pdf=True,
        pos=pos
    )

# Plot the covariances
_, ax = plt.subplots(1, 3, figsize=(10, 4), sharex=True, sharey=True)
samples = [neg_cov_res[1], zero_cov_res[1], pos_cov_res[1]]
titles = ["Negative Covariance", "Zero Covariance", "Positive Covariance"]

for i in range(3):
    ax[i].scatter(samples[i][:, 0], samples[i][:, 1], c="green")
    ax[i].set_xlabel("X1")
    ax[i].set_ylabel("X2")
    ax[i].set_title(titles[i])
    ax[i].tick_params(labelleft=True)
    ax[i].axvline(x=mu[0], color="blue", linestyle="--")
    ax[i].axhline(y=mu[1], color="red", linestyle="--")
plt.tight_layout()
plt.show()

The `blue` and `red` lines in the above plot represent the mean values of `x1` and `x2` respectively. Let's visualize how the probability density surface changes with the covariance

In [ ]:
fig = go.Figure(
    go.Surface(
        z=neg_cov_res[2],
        x=x1,
        y=x2,
        colorscale='Hot',
        showscale=False
    )
)
fig.update_layout(
    {
        "title": dict(
            text=f"Bivariate Distribution<br>cov_x1x2: {neg_cov[0][1]:.2f}",
            y=0.95,
            x=0.5,
            xanchor="center",
            yanchor="top",
            font=dict(size=12)
        ), 
        "scene":dict(
            xaxis=dict(title='X1'),
            yaxis=dict(title='X2'),
            zaxis=dict(title='Probability density')
        ),
        "xaxis": dict(title="Values"),
        "yaxis": dict(title="Probability Density"),
    }
)
fig.show()

In [ ]:
fig = go.Figure(
    go.Surface(
        z=zero_cov_res[2],
        x=x1,
        y=x2,
        colorscale='Viridis',
        showscale=False
    )
)
fig.update_layout(
    {
        "title": dict(
            text=f"Bivariate Distribution<br>cov_x1x2: {zero_cov[0][1]:.2f}",
            y=0.95,
            x=0.5,
            xanchor="center",
            yanchor="top",
            font=dict(size=12)
        ), 
        "scene":dict(
            xaxis=dict(title='X1'),
            yaxis=dict(title='X2'),
            zaxis=dict(title='Probability density')
        ),
        "xaxis": dict(title="Values"),
        "yaxis": dict(title="Probability Density"),
    }
)

fig.show()

In [ ]:
fig = go.Figure(
    go.Surface(
        z=pos_cov_res[2],
        x=x1,
        y=x2,
        showscale=False
    )
)
fig.update_layout(
    {
        "title": dict(
            text=f"Bivariate Distribution<br>cov_x1x2: {pos_cov[0][1]:.2f}",
            y=0.95,
            x=0.5,
            xanchor="center",
            yanchor="top",
            font=dict(size=12)
        ), 
        "scene":dict(
            xaxis=dict(title='X1'),
            yaxis=dict(title='X2'),
            zaxis=dict(title='Probability density')
        ),
        "xaxis": dict(title="Values"),
        "yaxis": dict(title="Probability Density"),
    }
)
fig.show()

A few things to note in the above surface plots:

1. The plot with zero covariance is circular in every direction.
2. The plots with negative and positive covariances are more flattened on the 45-degree line, and are somewhat in a perpendicular direction to that line visually.

# 3. Isotropic Gaussian

An isotropic Gaussian is one where the covariance matrix $\Sigma$ can be represented in this form:
$$
\Sigma = \sigma^2 I  \tag{4}
$$

1. $I$ is the Identity matrix
2. $\sigma^2$ is the scalar variance 

**Q**: Why do we want to represent the covarince matrix in this form?<br/>
**A**: As the dimensions of a multivariate Gaussian grows, the **mean** $\mu$ follows a linear growth where the **covarince matrix** $\Sigma$ follows quadratic growth in terms of number of parameters. This quadratic growth isn't computation friendly. A **diagonal covariance matrix** makes things much easier

A few things to note about isotropic Gaussian:

1. *Eq. (4)* represents a diagnal matrix multiplied by a scalar variance. This means that the **variance** along each dimesnion is equal. Hence an *isotropic multivariate Gaussian* is **circular** or **spherical**.

2. We discussed that `Cov(x1, x2)=0` doesn't mean `x1` and `x2` are independent but if the distribution is **multivariate normal** and `Cov(x1, x2)=0`, it implies that `x1` and `x2` are independent. 

3. If the multivariate distribution is isotropic, that means:
       i. Covariance Matrix is diagonal
       ii. The distribution can be represented as a product of univariate Gaussians i.e.
                P(X) = P(x1)P(x2)..
                
Let's take an example of an isotropic normal

In [ ]:
# Mean of the random variables X1 and X2
mu_x1, mu_x2 = 0, 0

# Standard deviation of the random variables X1 and X2
# Remember the std. is going to be the same
# along all the dimesnions.
sigma_x1 = sigma_x2 = 2

# Number of samples to extract
sample_size = 5000

# Positions for which probability density is
# to be generated
x1, x2 = np.mgrid[
    (-3.0 * sigma_x1 + mu_x1):(3.0 * sigma_x1 + mu_x1): 0.1,
    (-3.0 * sigma_x2 + mu_x2):(3.0 * sigma_x2 + mu_x2): 0.1
]
pos = np.dstack((x1, x2))

# Mean vector
mu = [mu_x1, mu_x2]


# Because the covariance matrix of an multivaraite isotropic
# gaussian is a diagonal matrix, hence the covariance for 
# the dimesnions will be zero.
cov_x1x2 = 0
cov = [[sigma_x1**2, cov_x1x2], [cov_x1x2, sigma_x2**2]]


isotropic_gaussian = get_multivariate_normal(
                        mu=mu,
                        cov=cov,
                        sample=True,
                        sample_size=sample_size,
                        seed=seed,
                        gen_pdf=True,
                        pos=pos
                    )

In [ ]:
fig = make_subplots(
    rows=1, cols=2,
    shared_yaxes=False,
    shared_xaxes=False,
    specs=[[{'type': 'scatter'}, {'type': 'surface'}]],
    subplot_titles=(
        "Covariance x1_x2 = 0.0",
        f"mu_x1: {mu_x1} sigma_x1: {sigma_x1**2} <br>mu_x2: {mu_x2} sigma_x2: {sigma_x2**2}"
    )
)


fig.add_trace(
    go.Scatter(
        x=isotropic_gaussian[1][:, 0],
        y=isotropic_gaussian[1][:, 1],
        mode='markers',
        marker=dict(size=5, color="green"),
    ),
    row=1, col=1
)

fig.add_trace(
    go.Surface(
        z=isotropic_gaussian[2],
        x=x1,
        y=x2,
        colorscale='RdBu',
        showscale=False
    ),
    row=1, col=2
)

fig.update_layout(
    {
        "scene":dict(
            xaxis=dict(title='X1'),
            yaxis=dict(title='X2'),
            zaxis=dict(title='Probability density')
        ),
        "xaxis": {"title": "X1"},
        "yaxis": {"title": "X2"},
        "title": {"text": "Isotropic Gaussian", "x":0.5, "font":dict(size=20)}
    }
)
fig.show()

# 4. Conditional Distribution 

Let's say we have a multivariate distribution over `x` where
$$
 x = \begin{bmatrix}x_{1}\\x_{2}\end{bmatrix} \\
 \mu = \begin{bmatrix}\mu_{1}\\\mu_{2}\end{bmatrix} \\
 \Sigma = \begin{bmatrix}\Sigma_{11} & \Sigma_{12} \\ \Sigma_{21} & \Sigma_{22}\end{bmatrix}
$$

Then the distribution of `x1` conditional on `x2=a` is multivariate normal:

$$
p(x1 | x2 = a) \sim N(\bar\mu, \bar\Sigma) \ \ \ \text  {where:}
$$

$$
\bar\mu = \mu_{1} + \ \Sigma_{12}\Sigma_{22}^{-1}(a - \mu_{2}) \tag{5}
$$
$$
\bar\Sigma = \Sigma_{11} - \ \Sigma_{12}\Sigma_{22}^{-1}\Sigma_{21} \tag{6}
$$

<br/>
<br/>

But why are we discussing the conditional distribution? Do you need to remember these equations?

Well, first and foremost you don't have to mug up any of these equations. You can find them easily on Wikipedia. But there is a reason why we are discussing the conditional distributions here.

Let's say you have a process that takes the form of a [**Markov Chain**](https://setosa.io/ev/markov-chains/). For example, the forward process in **Diffusion Models** is one example of such a sequence. Let's write down the equation for the same.
$$
q(x_{1:T}\vert x_{0}) := \prod_{t=1}^{T}q(x_{t}\vert x_{t-1}) :=\prod_{t=1}^{T}\mathcal{N}(x_{t};\sqrt{1-\beta_{t}} x_{t-1},\ \beta_{t}\bf I) \tag{7}
$$

This equation tells us a few things:

1. The forward process is a Markov Chain where the sample at the present step depends only on the sample at the previous timestep.
2. The covariance matrix is **diagonal**
3. At each timestep in this sequence, we gradually **add** Gaussian noise. But this isn't very clear from the equation where is this addition taking place, right?
4. The term $\beta$ represents the variance at a *particular timestep `t`* such that $0 < \beta_{1} < \beta_{2} < \beta_{3} < ...  < \beta_{T} < 1$

To understand the last point, let's make some assumptions to simplify the eq (7).<br/>
1. We will ignore the variance schedule $\beta$ for now. We can rewrite the above equation as: $$q(x_{1:T}\vert x_{0}) := \prod_{t=1}^{T}q(x_{t}\vert x_{t-1}) :=\prod_{t=1}^{T}\mathcal{N}(x_{t}; x_{t-1},\ \bf I)  \tag{8}$$
2. We will consider `x` as a univariate normal. This assumption is made only to give readers a better understanding of the conditional Gaussian case. The same concept extends to multivariate normal.

With the above assumptions in mind, combined with the `Law of total probability`, we can write the equation as:

$$p(x_{t}) = \int p(x_{t} \vert \ x_{t-1})p(x_{t-1})dx_{t-1} \tag {9}$$

Using *(8)*, we can rewrite this as:
$$p(x_{t}) = \int \mathcal{N}(x_{t}; \ x_{t-1}, 1) p(x_{t-1})dx_{t-1} \tag {10}$$

Because we are dealing with univariate normal, the identity matrix is nothing but a scalar value of 1. Moving forward, we can shift the terms in the above equation like this:
$$p(x_{t}) = \int \mathcal{N}(x_{t} - x_{t-1}; \ 0, 1) p(x_{t-1})dx_{t-1} \tag {11}$$

Notice two things that we got by this shift:
1. The mean and the variance values of the distribution on the right-hand side are `0` and `1` respectively.
2. If you look closely at the term on the RHS in the above equation, you will notice that it is the definition of **convolution**.

Combining the above two facts, we can rewrite *(11)* as:
$$p(x_{t}) = \mathcal{N}(0, 1) \ * \ p(x_{t-1}) \tag {12}$$

**Property**: The convolution of individual distributions of two or more random variables equals the sum of the random variables.

$$ \Longrightarrow x_{t} = \mathcal{N}(0, 1) \ + \ x_{t-1} \tag {13}$$

We will prove the above property in the next section, but this is one of the things that you should remember. Also, we hope that it is clear now why conditioned distributions in the forward process of Diffusion models are equivalent to "adding Gaussian noise" to previous timesteps. Don't worry about the diffusion models related equations and terms like `variance schedule`. We will talk about them in detail in the upcoming notebooks.

# 5. Convolution of probability distributions

In the last section we stated that the convolution of individual distributions of two or more random variables equals the sum of the random variables. To prove this, we will take an example of two normally distributed random variables `X` and `Y`. If `X` and `Y` are two normally distributed independent random variables such that

$$
X \sim \mathcal{N}(\mu_{1}, \sigma_{1}^2) \\
Y \sim \mathcal{N}(\mu_{2}, \sigma_{2}^2) \\
$$

$\text{if} \ Z = X + Y  \Longrightarrow Z \sim \mathcal{N}(\mu_{1} + \mu_{2}, \sigma_{1}^2 +\sigma_{2}^2) \tag{14}$ 

Our goal is to prove that if we convolve the PDFs of `X` and `Y`, then the resulting distribution would be identical to the distribution of `Z`. Let's write down the code for it.

In [ ]:
# Mean and Standard deviation of X
mu_x = 0.0
sigma_x = 2.0

# Mean and Standard deviation of Y
mu_y = 2.0
sigma_y = 4.0

# Mean and Standard deviation of Z
mu_z = mu_x + mu_y
sigma_z = np.sqrt(sigma_x**2 + sigma_y**2)

# Get the distributions
dist_x = norm(loc=mu_x, scale=sigma_x)
dist_y = norm(loc=mu_y, scale=sigma_y)
dist_z = norm(loc=mu_z, scale=sigma_z)

# Generate the PDFs
step_size = 1e-4
points = np.arange(-30, 30, step_size)

pdf_x = dist_x.pdf(points)
pdf_y = dist_y.pdf(points)
pdf_z = dist_z.pdf(points)


# NOTE: We cannot convolve over continous functions using `numpy.convolve(...)`
# Hence we will discretize our PDFs into PMFs using the step size we defined above
pmf_x = pdf_x * step_size
pmf_y = pdf_y * step_size

# Convolve the two PMFs
conv_pmf = np.convolve(pmf_x, pmf_y, mode="same")
conv_pdf = conv_pmf / step_size


# Let's plot the distributions now and check if we have gotten
# the same distribution as Z. 
# NOTE: As we have approximated PMF from PDF, there would be
# erros in the approximation. So, the final result may not
# look 100% identical.

_, ax = plt.subplots(1, 2, sharex=False, sharey=False, figsize=(12, 5))

ax[0].plot(points, pdf_x)
ax[0].plot(points, pdf_y)
ax[0].plot(points, pdf_z)
ax[0].set_title("Z = X + Y")
ax[0].legend(["PDF-X", "PDF-Y", "PDF-Z"])

ax[1].plot(points, pdf_x)
ax[1].plot(points, pdf_y)
ax[1].plot(points, pdf_z)
ax[1].set_title("Convolution of PDFs of X and Y")
ax[1].legend(["PDF-X", "PDF-Y", "Convolved"])

plt.show()

# 6. The Forward Process

Though we will talk about `Diffusion Models` in detail in the future posts as well, I will implement the forward process here to give you an idea that things that might look complicated in symbols aren't that complicated in terms of code.

Let's rewrite the forward process equation again:

$$
q(x_{1:T}\vert x_{0}) := \prod_{t=1}^{T}q(x_{t}\vert x_{t-1}) :=\prod_{t=1}^{T}\mathcal{N}(x_{t};\sqrt{1-\beta_{t}} x_{t-1},\ \beta_{t}\bf I)
$$

To implement the above equation:

1. We have to define the total number of timesteps `T`
2. We have to generate $\beta_{t}$ using a schedule. We can use any schedule including but not limited to linear, quadratic, etc. The only thing that we need to ensure is that $\beta_{1} < \beta_{2}...$
3. Sample a new image at timestep `t` from a conditional Gaussian for which the paramters are $\mu_{t} = \sqrt{1-\beta_{t}} x_{t-1}$ and $\sigma_{t}^2 = \beta_{t}$
4. For the last point, we can use the property(eq. (13)) we studied in the conditional distribution section. Hence we can write this as:

$$x_{t} \sim (\mathcal{N}(\sqrt{1-\beta_{t}} x_{t-1},\ \beta_{t}) + \mathcal{N}(0, 1))$$

$$
\Rightarrow x_{t} = \sqrt{1-\beta_{t}} x_{t-1} + \sqrt{\beta_{t}}\epsilon \ \ ; \ \text{where} \ \epsilon \sim \mathcal{N}(0, 1) \tag{15}
$$

Now that we have broken down the equation in much simpler parts, let's code it!

In [ ]:
def forward_process_ddpms(img_t_minus_1, beta, t):
    """Implements the forward process of a DDPM model.
    
    Args:
        img_t_minus_1: Image at the previous timestep (t - 1)
        beta: Scheduled Variance
        t: Current timestep
    Returns:
        Image obtained at current timestep
    """
    
    # 1. Obtain beta_t. Reshape it to have the same number of
    # dimensions as our image array
    beta_t = beta[t].reshape(-1, 1, 1)
    
    # 2. Calculate mean and variance
    mu = np.sqrt((1.0 - beta_t)) * img_t_minus_1
    sigma = np.sqrt(beta_t)
    
    # 3. Obtain image at timestep t using equation (15)
    img_t = mu + sigma * np.random.randn(*img_t_minus_1.shape)
    return img_t


# Let's download a sample image and check if our
# forward process function is doing what it is supposed to do
urllib.request.urlretrieve(
    "https://upload.wikimedia.org/wikipedia/commons/1/18/Dog_Breeds.jpg",
    "dog.jpg"
)

# 1. Load image using PIL (or any other library that you prefer)
img = Image.open("dog.jpg")

# 2. Resize the image to desired dimensions
IMG_SIZE = (128, 128)
img = img.resize(size=IMG_SIZE)

# 3. Define number of timesteps
timesteps = 100

# 4. Generate beta (variance schedule)
beta_start = 0.0001
beta_end = 0.05
beta = np.linspace(beta_start, beta_end, num=timesteps, dtype=np.float32)


processed_images = []
img_t = np.asarray(img.copy(), dtype=np.float32) / 255.

# 5. Run the forward process to obtain img after t timesteps
for t in range(timesteps):
    img_t = forward_process_ddpms(img_t_minus_1=img_t, beta=beta, t=t)
    if t%20==0 or t==timesteps - 1:
        sample = (img_t.clip(0, 1) * 255.0).astype(np.uint8)
        processed_images.append(sample)

# 6. Plot and see samples at different timesteps
_, ax = plt.subplots(1, len(processed_images), figsize=(12, 5))

for i, sample in enumerate(processed_images):
    ax[i].imshow(sample)
    ax[i].set_title(f"Timestep: {i*20}")
    ax[i].axis("off")
    ax[i].grid(False)

plt.suptitle("Forward process in DDPMs", y=0.75)
plt.show()

One thing to note here is that as you increase the number of timesteps `T`, $\beta_{t} \to 1$. At that point:

$$q(x_{T}) \approx \mathcal{N}(x_{T};\ 0, I)$$


That's it for now. We will talk about reverse process and other things related to DDPMs in the future notebooks. I hope this notebook was enough to give you a solid understanding of **Gaussian distribution** and their usage in context of DDPMs. Also, if you liked this work, please show your support by giving an upvote!


# References:

* https://en.wikipedia.org/wiki/Normal_distribution
* https://en.wikipedia.org/wiki/Multivariate_normal_distribution#Likelihood_function
* https://en.wikipedia.org/wiki/Convolution_of_probability_distributions
* https://online.stat.psu.edu/stat505/lesson/6/6.1
* https://www.inf.ed.ac.uk/teaching/courses/pmr/slides/cts4up.pdf
* http://timstaley.co.uk/posts/convolving-pdfs-in-python/
* https://www.assemblyai.com/blog/diffusion-models-for-machine-learning-introduction/
* https://www.youtube.com/watch?v=fbLgFrlTnGU&t=317s
* https://people.eecs.berkeley.edu/~jrs/189s17/lec/09.pdf
* https://mmuratarat.github.io/2019-10-05/univariate-multivariate_gaussian
* https://math.stackexchange.com/questions/4492913/isotropic-gaussian
* https://cs229.stanford.edu/section/gaussians.pdf
* https://math.stackexchange.com/questions/1991961/gaussian-distribution-is-isotropic
* https://www.cse.psu.edu/~rtc12/CSE586Spring2010/papers/prmlGaussian.pdf
* https://numpy.org/doc/stable/reference/random/generated/numpy.random.multivariate_normal.html
* https://docs.scipy.org/doc/scipy/tutorial/stats.html
* https://www.deeplearningbook.org/contents/prob.html
* https://arxiv.org/abs/2006.11239

In [ ]:
import os
import scipy
import urllib
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

seed = 1234
np.random.seed(seed)
plt.style.use("seaborn")

%config IPCompleter.use_jedi = False
%config InlineBackend.figure_format='png'
%matplotlib inline

# 1. The Forward Process

$$
q(x_{1:T}\vert x_{0})
:= \prod_{t=1}^{T}q(x_{t}\vert x_{t-1})
:=\prod_{t=1}^{T}\mathcal{N}(x_{t};\sqrt{1-\beta_{t}} x_{t-1},\ \beta_{t}\bf I) \tag{1}
$$

As a refresher, $q(x_{0:T})$ is known as the **forward distribution** and $q(x_{t}\vert x_{t-1}$ is referred as **forward diffusion kernel**

In [ ]:
def forward_process_ddpms(img_t_minus_1, beta, t):
    """Implements the forward process of a DDPM model.
    
    Args:
        img_t_minus_1: Image at the previous timestep (t - 1)
        beta: Scheduled Variance
        t: Current timestep
    Returns:
        Image obtained at current timestep
    """
    
    # 1. Obtain beta_t. Reshape it to have the same number of
    # dimensions as our image array
    beta_t = beta[t].reshape(-1, 1, 1)
    
    # 2. Calculate mean and variance
    mu = np.sqrt((1.0 - beta_t)) * img_t_minus_1
    sigma = np.sqrt(beta_t)
    
    # 3. Obtain image at timestep t using equation (15)
    img_t = mu + sigma * np.random.randn(*img_t_minus_1.shape)
    return img_t


# Let's check if ourforward process function is
# doing what it is supposed to do on a sample image
urllib.request.urlretrieve(
    "https://upload.wikimedia.org/wikipedia/commons/1/18/Dog_Breeds.jpg",
    "dog.jpg"
)

# 1. Load image using PIL (or any other library that you prefer)
img = Image.open("dog.jpg")

# 2. Resize the image to desired dimensions
IMG_SIZE = (128, 128)
img = img.resize(size=IMG_SIZE)

# 3. Define number of timesteps
timesteps = 100

# 4. Generate beta (variance schedule)
beta_start = 0.0001
beta_end = 0.05
beta = np.linspace(beta_start, beta_end, num=timesteps, dtype=np.float32)


processed_images = []
img_t = np.asarray(img.copy(), dtype=np.float32) / 255.

# 5. Run the forward process to obtain img after t timesteps
for t in range(timesteps):
    img_t = forward_process_ddpms(img_t_minus_1=img_t, beta=beta, t=t)
    if t%20==0 or t==timesteps - 1:
        sample = (img_t.clip(0, 1) * 255.0).astype(np.uint8)
        processed_images.append(sample)

# 6. Plot and see samples at different timesteps
_, ax = plt.subplots(1, len(processed_images), figsize=(15, 6))

for i, sample in enumerate(processed_images):
    ax[i].imshow(sample)
    ax[i].set_title(f"Timestep: {i*20}")
    ax[i].axis("off")
    ax[i].grid(False)

plt.suptitle("Forward process in DDPMs", y=0.75)
plt.show()
plt.close()

Can you spot a major problem with the above equation? (Hint: Check the loop)

Don't worry if you didn't get it. Look at the above code closely. Forget everything from the modelling perspective except for the forward pass. You will notice that to obtain a noisy sample, say at timestep `t`, we need to iterate from `t0` to `t-1`. Why? Because the sample obtained at each timestep is conditioned on the samples from the previous timesteps. 

That's not efficient. What if there are 1000 steps and you want to sample the `999th` timestep? You will iterate the whole loop, simulating the entire Markov Chain. Now that we know the problem, we should think about how we can do better.



# 2. Reparameterization

We know that sum of the independent Gaussians is still a Gaussian. We can leverage this fact to sample from an arbitrary forward step. All we need to do is apply the `reparameterization` trick.

Let &nbsp; $\alpha_{t} = 1 - \beta_{t},$ &nbsp; and  &nbsp; $\bar{\alpha}_{t} = \prod_{i=1}^T \alpha_{i}$

From equation (1) we know that:


$$
q(x_{1:T}\vert x_{0}) := \prod_{t=1}^{T}q(x_{t}\vert x_{t-1}) :=\prod_{t=1}^{T}\mathcal{N}(x_{t};\sqrt{1-\beta_{t}} x_{t-1},\ \beta_{t} \bf{I}) \\
$$

$$
\text{or} \ \ q(x_{t}\vert x_{t-1}) = \mathcal{N}(x_{t};\sqrt{1-\beta_{t}} x_{t-1},\ \beta_{t} \bf{I})
$$

We can obtain the sample at timestep `t` as:

$$
x_{t} = \sqrt{1 - \beta_{t}} x_{t-1} +  \sqrt{\beta_{t}}\epsilon_{t-1}; \ \ \text where \ \  \epsilon_{t-1} \sim \mathcal{N}(0, \bf{I})
$$

Replacing $\beta$ with $\alpha$ in the above equation we can obtain sample at timestep `t` as:

$$
\begin{align*}
x_{t} &= \sqrt{\alpha_{t}} x_{t-1} + \sqrt{1 - \alpha_{t}}\epsilon_{t-1} \\
\Rightarrow x_{t} &= 
\sqrt{\alpha_{t}} \ \ \underbrace{(\sqrt{\alpha_{t-1}} x_{t-2} + \sqrt{1 - \alpha_{t-1}}\epsilon_{t-2})}_{\text{( Expanding } x_{t-1})} + 
\sqrt{1 - \alpha_{t}}\epsilon_{t-1} \\ \\
&= \sqrt{\alpha_{t} \alpha_{t-1}} x_{t-2} + 
\underbrace{\sqrt{\alpha_{t}(1 - \alpha_{t-1})}\epsilon_{t-2}}_{\text{RV1}} + 
\underbrace{\sqrt{1 - \alpha_{t}}\epsilon_{t-1}}_{\text{RV2}} \\
\end{align*}
$$

The two terms namely RV1, and RV2 on RHS in the above equation are two random variables distributed **normally** with a mean of zero and variances $\alpha_{t}(1 - \alpha_{t-1})\ $, and $(1 - \alpha_{t})$ respectively. 

In the last [lesson](https://www.kaggle.com/code/aakashnain/ddpms-part-1) we learned that if we have two Gaussian distributions with mean values $\mu_{1} , \mu_{2}$ and variances $\sigma_{1}^2 , \sigma_{2}^2$ respectively, then the sum of these two random variables is equivaluent to another random variable with a normal distribution $\mathcal{N}(\mu_{1} + \mu_{2}, \sigma_{1}^2 +\sigma_{2}^2)$. Applying this to the above equation yields:

$$
\begin{align*}
\Rightarrow x_{t} &= 
\sqrt{\alpha_{t} \alpha_{t-1}} x_{t-2} + 
\sqrt{\alpha_{t}(1 - \alpha_{t-1}) + 
(1 - \alpha_{t})}\bar{z}_{t-2} & \bar{z}_{t-2} \ \text {is the merged Gaussian} \\
&= \sqrt{\alpha_{t} \alpha_{t-1}} x_{t-2} + 
\sqrt{1 - \alpha_{t} \alpha_{t-1}}\bar{z}_{t-2} \\
&= \ \ ... \\
\Rightarrow x_{t}&= \sqrt{\bar{\alpha_{t}}} x_{0} + 
\sqrt{1 - \bar{\alpha_{t}}}\epsilon \ \  \ \text{ (since } \
\ \bar{\alpha}_{t} = \prod_{i=1}^T \alpha_{i})
\end{align*}
$$

From above, we can say that:

$$
q(x_{t}\vert x_{0}) = \mathcal{N}(x_{t};\sqrt{\bar{\alpha_{t}}} x_{0},\ (1 - \bar{\alpha_{t}}) \ \bf{I}) \tag {2}
$$

Ha! The above equation is nice.Given the original image, we can now sample at any arbitrary timestep without simulating the entire Markov chain till that step. Before coding it up, let's recap of what we did and achieve:

1. We figured out that in the original formulation, we need to simulate the Markov chain to the step for which we want to sample.
2. We reparameterized $\beta$ in terms of $\alpha$
3. The above reparameterization leads to an equation where we can sample at any arbitrary timestep 


Let's code it and compare the results with the previous results 

In [ ]:
def forward_process_ddpms_v2(orig_img, alpha_bar, t):
    """Implements the efficient forward process of a DDPM model.
    
    Args:
        orig_img: Image at timestep t=0
        alpha_bar: The reparameterized version of beta
        t: Current timestep
    Returns:
        Image obtained at current timestep
    """
    
    # 1. Obtain beta_t. Reshape it to have the same number of
    # dimensions as our image array
    alpha_bar_t = alpha_bar[t].reshape(-1, 1, 1)
    
    # 2. Calculate mean and variance
    mu = np.sqrt(alpha_bar_t) * orig_img
    sigma = np.sqrt(1.0 - alpha_bar_t)
    
    # 3. Obtain image at timestep t
    img_t = mu + sigma * np.random.randn(*orig_img.shape)
    return img_t



# 1. Define alpha and alpha_bar
alpha = 1.0 - beta
alpha_bar = np.cumprod(alpha)


processed_images = [img] # Image at 0th step
orig_img = np.asarray(img.copy(), dtype=np.float32) / 255.


# 2. Run the forward pass for specific timesteps
# We will use the timesteps we used in previous visualizations
specific_timesteps = [19, 39, 59, 79, 99]
for step in specific_timesteps:
    img_t = forward_process_ddpms_v2(orig_img, alpha_bar, step)
    img_t = (img_t.clip(0, 1) * 255.0).astype(np.uint8)
    processed_images.append(img_t)

    
# 3. Plot and see samples at different timesteps
_, ax = plt.subplots(1, len(processed_images), figsize=(15, 6))

for i, sample in enumerate(processed_images):
    ax[i].imshow(sample)
    ax[i].set_title(f"Timestep: {i*20}")
    ax[i].axis("off")
    ax[i].grid(False)

plt.suptitle("Efficient Forward process in DDPMs", y=0.75)
plt.show()
plt.close()

Now that we are aware of everything involved in the forward process, we need to figure out how are we going to generate image from the noise we got at the end of the forward process. 


# 2. The Reverse Process

We should end up with a pure noise distribution by the end of the forward process, given we set the variance schedule appropriately i.e. the distribution we will end up with will be $\sim \mathcal{N}(x_{T}; 0,I)$. For the reverse process, we will start with noise, and will try to undo the noise at each timestep to obtain back the original image. We can write this process as:

$$
p_{\theta}(x_{0:T}) 
:= p(x_{T}) \prod_{t=1}^T p_{\theta}(x_{t-1} | x_{t}) 
:= p(x_{T}) \prod_{t=1}^T \mathcal{N}(x_{t-1}; \mu_{\theta}(x_{t}, t), \Sigma_{\theta}(x_{t}, t)) \tag{3}
$$

where the parameters of the multivariate Gaussian are time-dependent and are to be learned.

A few things to note:

1. $p(x_{0:T})$ is the **reverse distribution** and $p(x_{t-1} | x_{t})$ is known as the **reverse diffusion kernel**
2. $\mu_{\theta}(x_{t}, t), \Sigma_{\theta}(x_{t}$ are the learnable parameters of the reverse distribution
3. The forward process can be seen as pushing the sample off the data mainfold, turning it into noise. The reverse process can be seen as pushing the sample back to the manifold by removing the noise. (Words taken from Ari Seff's [tutorial](https://www.youtube.com/watch?v=fbLgFrlTnGU) because I don't think there is any better way to put it.)
4. $p(x_{T})$ is nothing but $q(x_{T})$ i.e. the point where the forward process ends is the starting point of the reverse process.
5. There can be `n` number of pathways to arrive at a sample $p_{\theta}(x_{0})$ starting from a noise sample. To obtain $p_{\theta}(x_{0})$, we would then be required to integrate over all the possible pathways i.e. $p_{\theta}(x_{0}) = \int p_{\theta}(x_{0:T})dx_{1:T}$
6. Calculating density like in the above step is intractable. A neural network is sufficient to predict the mean $\mu_{\theta}$ and the diagonal covariance matrix $\Sigma_{\theta}$ for the reverse process as shown below in the equation, but we would also be required to frame our objectve function differently



# 3. The Training Objective

Let's write down all the equations we saw for the reverse process, again before we move to the discussion on training objective. 

$$
\begin{align*}
p_{\theta}(x_{0}) &= \int p_{\theta}(x_{0:T})dx_{1:T} \tag{4} \\
p_{\theta}(x_{t-1}|x_{t}) &:= \mathcal{N}(x_{t-1}; \mu_{\theta}(x_{t}, t), \Sigma_{\theta}(x_{t}, t)) \tag{5}
\end{align*}
$$

If eq.*(4)* is intractable, then how do we frame our objective function? 

If you have worked with other types of generative models before, then you might have seen something like equation eq.*(4)* in other types of generative models. But where? The answer is **Variational AutoEncoder**(VAE for short). If we treat $x_{0}$ as **observed variable** and $x_{1:T}$ as **latent variables**, then we have a setup similar to a VAE.

Similarities:
1. The forward process can be seen as the equivalent of the encoder in a VAE converting data to latents
2. The reverse process can be seen as the equivalent of the decoder in a VAE producing data from latents
3. Given that the above two hold, we can maximize the lower bound (Check out this [post](https://lilianweng.github.io/posts/2018-08-12-vae/#loss-function-elbo) if you need a referesher on ELBO)

Differences:
1. Unlike a VAE where the encoder is jointly trained with the decoder, the forward process in DDPMs is fixed. Only the reverse part is trainable
2. Unlike in VAEs, the latents in DDPMs have the same dimensionality as the observed variable
3. Variation lower bound or Evidence lower bound (ELBO for short), in the case of DDPMs, is a sum of losses at each time step `t`, $L = L_{0} + L_{1} + ... + L_{T}$ <br/> 

First, let's write down the lower bound we see in a VAE here. Let's say **x** is an observed variable, and **z** is the latent variable (in context of a VAE). It is known that:

$$
\begin{align*}
&\text{log}\ p_{\theta}(x) \ge \text{variational lower bound} \\
\Rightarrow &\text{log}\ p_{\theta}(x) \ge \mathbb{E}_{q_{\phi}(z|x)}[\text{log}\ p_{\theta}(x|z)] \ - \ D_{KL}(q_{\phi}(z|x) \parallel	p_{\theta}(z)) \tag{6}
\end{align*}
$$


As discussed earlier, when it comes to diffusion models, we can treat $x_{0}$ as the observed variable, and $x_{1:T}$ as the latent varaibles. Substituting these in equation *(6)* yields:

$$
\begin{align*}
\text{log}\ p_{\theta}(x_{0}) &\ge \mathbb{E}_{q(x_{1:T}|x_{0})}[\text{log}\ p_{\theta}(x_{0}|x_{1:T})] \ - \ \underbrace{D_{KL}(q(x_{1:T}|x_{0}) \parallel p_{\theta}(x_{1:T}))} \\ \\ 
&=\mathbb{E}_{q(x_{1:T}|x_{0})}[\text{log}\ p_{\theta}(x_{0}|x_{1:T})] \ - \ \mathbb{E}_{q(x_{1:T}|x_{0})}\big[\text{log}\frac{q(x_{1:T}|x_{0})}{p_{\theta}(x_{1:T})}\big] \\
&=\mathbb{E}_{q}[\text{log}\ p_{\theta}(x_{0}|x_{1:T})] \ - \ 
\mathbb{E}_{q} \big[ \text{log}\frac{q(x_{1:T}|x_{0})}{p_{\theta}(x_{1:T})} \big]
\\
&=\mathbb{E}_{q}\bigg[\text{log}\ p_{\theta}(x_{0}|x_{1:T}) \ - \ 
\text{log}\frac{q(x_{1:T}|x_{0})}{p_{\theta}(x_{1:T})}\bigg]
\\
&=\mathbb{E}_{q}\biggl[\text{log}\ p_{\theta}(x_{0}|x_{1:T}) \ + \ \text{log}\frac{p_{\theta}(x_{1:T})}{q(x_{1:T}|x_{0})}\biggr]
\\
&=\mathbb{E}_{q}\biggl[\text{log}\ \frac{p_{\theta}(x_{0}|x_{1:T}) \ p_{\theta}(x_{1:T})}{{q(x_{1:T}|x_{0})}}\biggr]
\\
&=\mathbb{E}_{q}\biggl[\text{log}\frac{p_{\theta}(x_{0:T})}{q(x_{1:T}|x_{0})}\biggr] \tag{7} \\
\end{align*}
$$


The maths above looks scary but if you look closely, we haven't done anything fancy apart from applying standard definitions of expectation, KL-Divergence, and log to the original equation. Let's denote equation *(7)* by ${L}$

$$
\begin{align*}
{L} &=\mathbb{E}_{q}\biggl[\text{log}\frac{p_{\theta}(x_{0:T})}{q(x_{1:T}|x_{0})}\biggr] \\
&= \mathbb{E}_{q}\biggl[\text{log}\frac{p(x_{T})\prod_{t=1}^T p_{\theta}(x_{t-1}|x_{t})}{\prod_{t=1}^T q(x_{t}|x_{t-1})}\biggr] \\
\Rightarrow{L} &= \mathbb{E}_{q}\biggl[\text{log}\ p(x_{T}) \ + \ \underbrace{\sum\limits_{t=1}^{T}\text{log}\frac{p_{\theta}(x_{t-1}|x_{t})}{q(x_{t}|x_{t-1})}}\biggr]
\end{align*}
$$


Using Bayes' rule, we know that,
$$
\begin{align*}
q(x_{t-1} | x_{t}, x_{0}) &= q(x_{t} | x_{t-1}, x_{0}) \frac{q(x_{t-1}| x_{0})}{q(x_{t} | x_{0})} \tag{8}
\end{align*}
$$

$$
\begin{align*}
{L} &= \mathbb{E}_{q}\biggl[\text{log}\ p(x_{T}) \ + \ \underbrace{\sum\limits_{t=1}^{T}\text{log}\frac{p_{\theta}(x_{t-1}|x_{t})}{q(x_{t}|x_{t-1})}}\biggr] \\
\text{Using (8) we can rewrite this as:} \\
L &= \mathbb{E}_{q}\biggl[\text{log}\ p(x_{T}) \ + \ \sum\limits_{t=2}^{T}\text{log}\frac{p_{\theta}(x_{t-1}|x_{t})}{q(x_{t}|x_{t-1})} \ + \ \text{log}\frac{p_{\theta}(x_{0}|x_{1})}{q(x_{1}|x_{0})}\biggr] \\
\Rightarrow L &= \mathbb{E}_{q}\biggl[\text{log}\ p(x_{T}) \ + \ \text{log}\frac{p_{\theta}(x_{0}|x_{1})}{q(x_{1}|x_{0})} \ + \ \sum\limits_{t=2}^{T}\text{log}\frac{q(x_{t-1}|x_{0})}{q(x_{t}|x_{0})}\ + \ \sum\limits_{t=2}^{T}\text{log}\frac{p_{\theta}(x_{t-1}|x_{t})}{q(x_{t-1}|x_{t}, x_{0})}\biggr] \\
\Rightarrow L &= \mathbb{E}_{q}\biggl[\text{log}\ p(x_{T}) \ + \ \text{log}\frac{p_{\theta}(x_{0}|x_{1})}{q(x_{1}|x_{0})} \ + \ \underbrace{\sum\limits_{t=2}^{T}\text{log}\bigl(q(x_{t-1}|x_{0})\bigr) \ - \ \text{log}\bigl({q(x_{t}|x_{0})\bigr)}} \ + \ \sum\limits_{t=2}^{T}\text{log}\frac{p_{\theta}(x_{t-1}|x_{t})}{q(x_{t-1}|x_{t}, x_{0})}\biggr] \\
\text{Expanding that summation, we get:} \\
L &= \mathbb{E}_{q}\biggl[\text{log}\ p(x_{T}) \ + \ \text{log}\frac{p_{\theta}(x_{0}|x_{1})}{q(x_{1}|x_{0})} \ + \ \text{log}\frac{q(x_{1} | x_{0})}{q(x_{T}|x_{0})} \ + \ \sum\limits_{t=2}^{T}\text{log}\frac{p_{\theta}(x_{t-1}|x_{t})}{q(x_{t-1}|x_{t}, x_{0})}\biggr] \\
&= \mathbb{E}_{q}\biggl[\text{log}\ p(x_{T}) \ + \ \text{log }{p_{\theta}(x_{0}|x_{1})\ - \ \text{log }q(x_{1}|x_{0})} \ + \ \text{log }q(x_{1} | x_{0}) \ - \ \text{log }{q(x_{T}|x_{0})} \ + \ \sum\limits_{t=2}^{T}\text{log}\frac{p_{\theta}(x_{t-1}|x_{t})}{q(x_{t-1}|x_{t}, x_{0})}\biggr] \\
&= \mathbb{E}_{q(x_{1:T}|x_{0})}\biggl[\text{log} \frac{p(x_{T})}{q(x_{T}|x_{0})} \ + \ \text{log }{p_{\theta}(x_{0}|x_{1})} \ + \ \sum\limits_{t=2}^{T}\text{log}\frac{p_{\theta}(x_{t-1}|x_{t})}{q(x_{t-1}|x_{t}, x_{0})}\biggr] \\
\Rightarrow L &= \mathbb{E}_{q(x_{1}|x_{0})}\big[\text{log }{p_{\theta}(x_{0}|x_{1})}\big] \ + \ \mathbb{E}_{q(x_{T}|x_{0})}\big[\text{log }\frac{p(x_{T})}{q(x_{T}|x_{0})}\big] \ + \  \sum\limits_{t=2}^{T}\mathbb{E}_{q(x_{t},\ x_{t-1}|x_{0})}\bigg[\text{log}\frac{p_{\theta}(x_{t-1}|x_{t})}{q(x_{t-1}|x_{t}, x_{0})}\biggr] \tag{9}\\
\end{align*}
$$
<br/>


In equation *(9)* we used linearity of expectation to expand the terms and rearranged the subscripts accordingly. This has provided us an opportunity to express the expression `L` in terms of KL-divergence. Let's do that

$$
\begin{align*}
L &= \mathbb{E}_{q(x_{1}|x_{0})}\big[\text{log }{p_{\theta}(x_{0}|x_{1})}\big] \ + \ \mathbb{E}_{q(x_{T}|x_{0})}\big[\text{log }\frac{p(x_{T})}{q(x_{T}|x_{0})}\big] \ + \  \sum\limits_{t=2}^{T}\mathbb{E}_{q(x_{t},\ x_{t-1}|x_{0})}\bigg[\text{log}\frac{p_{\theta}(x_{t-1}|x_{t})}{q(x_{t-1}|x_{t}, x_{0})}\biggr] \\
\Rightarrow L &= \mathbb{E}_{q(x_{1}|x_{0})}\big[\text{log }{p_{\theta}(x_{0}|x_{1})}\big] \ - \ D_{KL}\bigg(q(x_{T}|x_{0}) \parallel \ p(x_{T}) \bigg) \ - \ \sum\limits_{t=2}^{T}\mathbb{E}_{q(x_{t}|x_{0})}\big[D_{KL}\big(q(x_{t-1}|x_{t}, x_{0}) \ \parallel \ p_{\theta}(x_{t-1}|x_{t}) \big] \tag{10} \\
\end{align*}
$$<br/>



A few points to note about this equation:

1. Take a look at the second term $D_{KL}\big(q(x_{T}|x_{0}) \parallel \ p(x_{T}\big)$. The forward process represented by `q` is fixed. The term $p(x_{T})$ is nothing but the end point of the forward process and the start of the reverse process, hence it is fixed as well. This means that we can safely ignore the second term in the optimization process. 
2. Let's take a look at the KL-divergence involved in the third term $D_{KL}\big(q(x_{t-1}|x_{t}, x_{0}) \ \parallel \ p_{\theta}(x_{t-1}|x_{t})$ This is known the *denoising matching term*. We proved earlier that if $x_{0}$ is known, we can show that the intermediate step in the forward process `q` are Gaussian (we will show it later). The reverse steps in `p` are also parameterized as Gaussian (check eq. (3)), hence we can say that the KL divergence term at each timestep is a comparison between two Gaussians.  The term $(q(x_{t-1}|x_{t}, x_{0})$ also serves as the GT since it defines how to denoise a noisy image ${x_{t}}$ with access to what the final, completely denoised image $x_{0}$ should be.
3. As a result of above two, we can rewrite our  training objective as:
$$
L = \mathbb{E}_{q(x_{1}|x_{0})}\big[\text{log }{p_{\theta}(x_{0}|x_{1})}\big] \ - \ \sum\limits_{t=2}^{T}\mathbb{E}_{q(x_{t}|x_{0})}\big[D_{KL}\big(q(x_{t-1}|x_{t}, x_{0}) \ \parallel \ p_{\theta}(x_{t-1}|x_{t}) \big] - C \tag{11} \\
$$

<br/>

Let's revisit equation (8) again now. We know that:

$$
\begin{align*}
q(x_{t-1} | x_{t}, x_{0}) &= q(x_{t} | x_{t-1}, x_{0}) \frac{q(x_{t-1}| x_{0})}{q(x_{t} | x_{0})}
\end{align*}
$$

As proved earlier, we can sample any arbritrary forward step given $x_{0}$ as:
$$
q(x_{t}\vert x_{0}) = \mathcal{N}(x_{t};\sqrt{\bar{\alpha_{t}}} x_{0},\ (1 - \bar{\alpha_{t}}) I)
$$

Combining the above two facts, we can say that:

$$
\begin{align*}
q(x_{t-1} | x_{t}, x_{0}) &= q(x_{t} | x_{t-1}, x_{0}) \frac{q(x_{t-1}| x_{0})}{q(x_{t} | x_{0})} \\
&= \frac{\mathcal{N}(x_{t};\sqrt{\bar{\alpha_{t}}} x_{0},\ (1 - \alpha_{t}) \ I) \ \ \mathcal{N}(x_{t-1};\sqrt{\bar{\alpha}_{t-1}} x_{0},\ (1 - \bar{\alpha}_{t-1}) \ I)}{\mathcal{N}(x_{t};\sqrt{\bar{\alpha_{t}}} x_{0},\ (1 - \bar{\alpha_{t}}) \ I)} \\
&\propto \text{exp} \bigg\{-\frac{1}{2}\biggl[
\frac{(x_{t} - \sqrt{\alpha_{t}}x_{t-1})^2}{(1 \ - \ \alpha_{t})} \ + \ 
\frac{(x_{t-1} - \sqrt{\bar{\alpha}_{t-1}}x_{0})^2}{(1 \ - \ \bar{\alpha}_{t-1})} \ - \
\frac{(x_{t-1} - \sqrt{\bar{\alpha}_{t}}x_{0})^2}{(1 \ - \ \bar{\alpha}_{t})}
\biggr]\bigg\} \\
&= \text{exp} \bigg\{-\frac{1}{2}\biggl[
\frac{(x_{t} - \sqrt{\alpha_{t}}x_{t-1})^2}{\beta_{t}} \ + \ 
\frac{(x_{t-1} - \sqrt{\bar{\alpha}_{t-1}}x_{0})^2}{(1 \ - \ \bar{\alpha}_{t-1})} \ - \
\frac{(x_{t-1} - \sqrt{\bar{\alpha}_{t}}x_{0})^2}{(1 \ - \ \bar{\alpha}_{t})}
\biggr]\bigg\} \\
&= \text{exp} \bigg\{-\frac{1}{2}\biggl[
\frac{x_{t}^2 - 2\sqrt{\alpha_{t}}x_{t-1}x_{t} \ + \ \alpha_{t}x_{t-1}^2}{\beta_{t}} \ + \ 
\frac{x_{t-1}^2 - 2\sqrt{\bar{\alpha}_{t-1}}x_{0}x_{t-1} \ + \ \bar{\alpha}_{t-1}x_{0}^2}{(1 \ - \ \bar{\alpha}_{t-1})} \ - \
\frac{(x_{t-1} - \sqrt{\bar{\alpha}_{t}}x_{0})^2}{(1 \ - \ \bar{\alpha}_{t})}
\biggr]\bigg\} \\
&= \text{exp} \bigg\{-\frac{1}{2}\biggl[
\bigg(\frac{\alpha_{t}}{\beta_{t}} \ + \ \frac{1}{1 \ - \ \bar{\alpha}_{t-1}}\bigg) x_{t-1}^2 \ - \
\bigg(\frac{2\sqrt{\alpha_{t}}\ x_{t}}{\beta_{t}} \ + \ \frac{2\sqrt{\bar{\alpha}_{t-1}}\ x_{0}}{1 \ - \ \bar{\alpha}_{t-1}}\bigg) x_{t-1} \ + \ C(x_{t}, x_{0}) 
\biggr]\bigg\} \tag{12} \\
\end{align*}
$$
<br/>

The term $C(x_{t}, x_{0}$ is some value computed using $x_{t}, x_{0}, \text{and} \ \alpha$, and doesn't involve $x_{t-1}$. Hence we treat it as a constant term w.r.t to $x_{t-1}$. We know that in the case of a standard Gaussian, the density is proportional to:

$$
\begin{align*}
&\propto \text{exp}\big(- \ \frac{(x - \mu)^2}{2\sigma^2}\big) \\
&=\text{exp}\big(- \frac{1}{2} \frac{(x^2 \ + \ \mu^2 \ - \ 2\mu x)}{\sigma^2}\big) \tag{13} \\
\end{align*}
$$
<br/>

Comparing the coefficients of our Gaussian in **(12)** and the standard Gaussian in **(13)** we get:

$$
\begin{align*}
\tilde{\beta_{t}} &=
1 \ \big/ \bigg(\frac{\alpha_{t}}{\beta_{t}} \ + \ \frac{1}{1 \ - \ \bar{\alpha}_{t-1}}\bigg) = 
\frac{1- \bar{\alpha}_{t-1}}{1- \bar{\alpha}_t} \beta_{t} 
= \frac{(1- \bar{\alpha}_{t-1})(1 - \alpha_{t})}{1 - \bar{\alpha}_{t}} \tag{14} \\ \\ \\
\tilde{\mu}(x_{t}, x_{0}) &= 
\frac
{
\dfrac{\sqrt{\alpha_{t}}\ x_{t}}{\beta_{t}} \ + \ \dfrac{\sqrt{\bar{\alpha}_{t-1}}\ x_{0}}{1 \ - \ \bar{\alpha}_{t-1}}
}
{
\dfrac{\alpha_{t}}{\beta_{t}} \ + \ \dfrac{1}{1 \ - \ \bar{\alpha}_{t-1}}
} \\ \\
&=\dfrac{\sqrt{\alpha_{t}}\ x_{t}}{\beta_{t}} \ + \ \dfrac{\sqrt{\bar{\alpha}_{t-1}}\ x_{0}}{1 \ - \ \bar{\alpha}_{t-1}} \\ \\
&=\bigg(
\dfrac{\sqrt{\alpha_{t}}\ x_{t}}{\beta_{t}} \ + \ \dfrac{\sqrt{\bar{\alpha}_{t-1}}\ x_{0}}{1 \ - \ \bar{\alpha}_{t-1}}
\bigg)
\frac{1- \bar{\alpha}_{t-1}}{1- \bar{\alpha}_t} \beta_{t} \\ \\
\Rightarrow \tilde{\mu}(x_{t}, x_{0}) &= 
\frac{\sqrt{\alpha_{t}}(1- \bar{\alpha}_{t-1})}{1- \bar{\alpha}_{t}} x_{t} \ + \ 
\frac{\sqrt{\alpha_{t-1}}\ \beta_{t}}{1- \bar{\alpha}_{t}} x_{0} \tag{15} \\
\end{align*}
$$
<br/>

When we applied the reparameterization trick, we learned that we can write:<br/>
$$
x_{t} =  \sqrt{\bar{\alpha_{t}}}x_{0} + \sqrt{1 - \bar{\alpha_{t}}}\epsilon_{t} \\
$$

Hence we can express *(15)* as:

$$
\begin{align*}
\Rightarrow \tilde{\mu}(x_{t}, x_{0}) &= 
\frac{\sqrt{\alpha_{t}}(1- \bar{\alpha}_{t-1})}{1- \bar{\alpha}_{t}} x_{t} \ + \
\frac{\sqrt{\alpha_{t-1}}\ \beta_{t}}{1- \bar{\alpha}_{t}}
\frac{1}{\sqrt{\bar{\alpha}}_{t}} (x_{t} \ - \ \sqrt{1 \ - \ \bar{\alpha}_{t}}\epsilon_{t}) \\
\Rightarrow \tilde{\mu}(x_{t}, x_{0}) &=
\frac{1}{\sqrt{\bar{\alpha}}_{t}} \bigg(x_{t} \ - \ \frac{\beta_{t}}{\sqrt{1 \ - \ \bar{\alpha}_{t}}}\epsilon_{t}\bigg)
\tag{16}
\\ \\
\therefore \ &q(x_{t-1} | x_{t}, x_{0}) \sim \mathcal {N}(x_{t-1}; \tilde{\mu}(x_{t}, x_{0}), \ \tilde{\beta}\bf{I})
\end{align*}
$$

So, $q(x_{t-1} | x_{t}, x_{0})$ is the true distribution, and $\tilde{\mu}(x_{t}, x_{0}),\text{, and } \tilde{\beta}$ are the true parameters we are trying to approximate.
Let's talk about the other distriution  $p_{\theta}(x_{t-1}, x_{t})$. Our model has to approximate the conditioned probability distributions in the reverse diffusion process:

$$
p_{\theta}(x_{t-1}|x_{t}) := \mathcal{N}(x_{t-1}; \mu_{\theta}(x_{t}, t), \Sigma_{\theta}(x_{t}, t))
$$

Though we can try to learn both the mean and the variance of this distribution, the authors of the DDPMs paper found that learning the variance leads to lower quality samples. They decided to keep the variance $\Sigma_{\theta}(x_{t}, t)$ to time specific constants. Hence our network is solely tasked with learning the mean $\mu_{\theta}(x_{t}, t)$. The true mean that we are trying to approximate is:

$$
\begin{align*}
\tilde{\mu}(x_{t}, x_{0}) &=
\frac{1}{\sqrt{\bar{\alpha}}_{t}} \bigg(x_{t} \ - \ \frac{\beta_{t}}{\sqrt{1 \ - \ \bar{\alpha}_{t}}}\epsilon_{t}\bigg) = 
\frac{1}{\sqrt{\bar{\alpha}}_{t}} \bigg(x_{t} \ - \ \frac{1 \ - \ \alpha_{t}}{\sqrt{1 \ - \ \bar{\alpha}_{t}}}\epsilon_{t}\bigg)
\end{align*}
$$

$x_{t}$ will be available as an input during training. $\alpha_{t}$ is already known to us in advance (check the Reparameterization section), therefore we can reparameterize the Gaussian noise term instead to predict $\epsilon_{t}$ at a given timestep `t` as:

$$
\begin{align*}
\mu_{\theta}(x_{t}, t) &=
\frac{1}{\sqrt{\bar{\alpha}}_{t}} \bigg(x_{t} \ - \ \frac{1 \ - \ \alpha_{t}}{\sqrt{1 \ - \ \bar{\alpha}_{t}}}\epsilon_{\theta}(x_{t}, t)\bigg) \tag{17}
\end{align*}
$$

Let's recap of all the things we covered in the last section:
1. We showed that $q(x_{t-1} | x_{t}, x_{0})$ is just a Gaussian, and that the KL-divergence at each timestep (for $t > 1$) between $q(x_{t-1} | x_{t}, x_{0})$ and $p_{\theta}(x_{t-1}|x_{t})$ is KL-divergence between two Gaussian distributions
2. During training, we have kept the variances of the reverse process to time-specific constants. Our only task is to learn to approximate the mean during training.
3. With reparameterization, we can instead learn to predict the noise at each timestep. 

<br/>

Let's revisit our objectve function now. Writing down the equation here:<br/>
$$
L = \mathbb{E}_{q(x_{1}|x_{0})}\big[\text{log }{p_{\theta}(x_{0}|x_{1})}\big] \ - \ \sum\limits_{t=2}^{T}\mathbb{E}_{q(x_{t}|x_{0})}\big[D_{KL}\big(q(x_{t-1}|x_{t}, x_{0}) \ \parallel \ p_{\theta}(x_{t-1}|x_{t}) \big] \ - C \\
$$

We know that the KL divergence between two Gaussian distributions can be evaluated in closed form (Check out this excellent [post](https://mr-easy.github.io/2020-04-16-kl-divergence-between-2-gaussian-distributions/) for more details). Also, the variance is fixed, and we only need to minimize the difference between the actual mean and the predicted mean. 

$$
\begin{align*}
L_{t\sim U \{ 2, T \} } = \mathbb{E}_{q}\big[D_{KL}\big(
\mathcal {N}(x_{t-1}; \tilde{\mu}(x_{t}, x_{0}), \ \tilde{\beta}{I}) \ \parallel \ 
\mathcal{N}(x_{t-1}; \mu_{\theta}(x_{t}, t), \Sigma_{\theta}(x_{t}, t)\big) \big] \\
\end{align*}
$$
<br/>

Focusing on the KL divergence part:<br/>
$$
\begin{align*}
&D_{KL}\big(
\mathcal {N}(x_{t-1}; \tilde{\mu}(x_{t}, x_{0}), \ \tilde{\beta}{I}) \ \parallel \ 
\mathcal{N}(x_{t-1}; \mu_{\theta}(x_{t}, t), \Sigma_{\theta}(x_{t}, t)\big) 
\\
&=
\frac{1}{2} \bigg[\text{log}\frac{\bar{\beta}}{\Sigma_{\theta}(x_{t}, t)} \ - \ d  \ + \
\big(\bar\mu(x_{t}, x_{0}) \ - \  \mu_{\theta}(x_{t}, t)\big)^{T} \Sigma_{\theta}(x_{t}, t)^{-1} \big(\bar\mu(x_{t}, x_{0}) \ - \  \mu_{\theta}(x_{t}, t)\big)\big)\bigg]
\\
\because\text{variances are kept fixed}
\\
&=\frac{1}{2} \bigg[\text{log}\frac{\bar{\beta}}{\bar{\beta}} \ - \ d  \ + \
\text{tr}\big({\bar{\beta}^{-1} {\bar{\beta}}}\big) \ + \
\big(\bar\mu(x_{t}, x_{0}) \ - \  \mu_{\theta}(x_{t}, t)\big)^{T} \bar{\beta}^{-1} \big(\bar\mu(x_{t}, x_{0}) \ - \  \mu_{\theta}(x_{t}, t)\big)\big)\bigg]  \ \ \ \ \ 
\\
&=\frac{1}{2} \bigg[\text{log}1 \ - \ d  \ + \ d \ + \
\big(\bar\mu(x_{t}, x_{0}) \ - \  \mu_{\theta}(x_{t}, t)\big)^{T} \bar{\beta}^{-1} \big(\bar\mu(x_{t}, x_{0}) \ - \  \mu_{\theta}(x_{t}, t)\big)\big)\bigg]
\\
&=\frac{1}{2\bar{\beta}} \bigg[
\big(\bar\mu(x_{t}, x_{0}) \ - \  \mu_{\theta}(x_{t}, t)\big)^{T} \big(\bar\mu(x_{t}, x_{0}) \ - \  \mu_{\theta}(x_{t}, t)\big)\big)\bigg]
\end{align*}
$$
<br/><br/>

Now that we have expanded the KL divergence, we can rewrite our loss function as: <br/>

$$
\begin{align*}
\newcommand{\norm}[1]{\left\lVert #1 \right\rVert}
L_{t\sim U \{ 2, T \} } &= \mathbb{E}_{x_{0}, \epsilon, t}\bigg[
\dfrac{1}{2\bar{\beta}}
\norm{\tilde{\mu}_{\theta}(x_{t}, x_{0})\ - \ \mu_{\theta}(x_{t}, t)}^2
\bigg] \\
&=\mathbb{E}_{x_{0}, \epsilon, t}\bigg[
\dfrac{1}{2 \bar{\beta}}
\norm{
\frac{1}{\sqrt{\bar{\alpha}}_{t}} \bigg(x_{t} \ - \ 
\frac{1 \ - \ \alpha_{t}}{\sqrt{1 \ - \ \bar{\alpha}_{t}}}\epsilon_{t})\bigg) \ - \
\frac{1}{\sqrt{\bar{\alpha}}_{t}} \bigg(x_{t} \ - \ 
\frac{1 \ - \ \alpha_{t}}{\sqrt{1 \ - \ \bar{\alpha}_{t}}}\epsilon_{\theta}(x_{t}, t)\bigg)
}^2
\bigg] \\
&=\mathbb{E}_{x_{0}, \epsilon, t}\bigg[
\underbrace{\dfrac{(1 \ - \ \alpha_{t})^2}{2 \alpha_{t} (1 \ - \ \alpha_{t})\bar{\beta}}}_{\text{step-specific weighting}}
\norm{\epsilon_t \ - \ \epsilon_{\theta}(x_{t}, t)}^2
\bigg] \tag{18}
\end{align*}
$$
<br/>

<br/> A simpler version was also proposed by the authors, where they also discard this step-specific weigthing term. We can rewrite our final loss function as: <br/>

$$
\begin{align*}
L^{simple} &= \mathbb{E}_{x_{0}, \epsilon, t}\big[
\norm{\epsilon_t \ - \ \epsilon_{\theta}(x_{t}, t)}^2
\big]
\end{align*}
$$

<br/> The final objective function can be written as: <br/>
$$
\begin{align*}
L = L^{simple} \  +  \ C \tag{19}
\end{align*}
$$

where **C** is a constant not depending on $\theta$

That's it for now. I hope this notebook was enough to give you a solid understanding of the Diffusion models and the underlying maths. I tried to break down the maths as much as possible. If you liked this work, please consider giving an **upvote**!


# References

1. https://blog.evjang.com/2016/08/variational-bayes.html
2. https://lilianweng.github.io/posts/2018-08-12-vae/
3. https://lilianweng.github.io/posts/2021-07-11-diffusion-models/
4. https://www.youtube.com/watch?v=fbLgFrlTnGU
5. https://arxiv.org/pdf/2208.11970.pdf
6. https://www.assemblyai.com/blog/diffusion-models-for-machine-learning-introduction/
7. https://mr-easy.github.io/2020-04-16-kl-divergence-between-2-gaussian-distributions/